In [ ]:
import opensim as osim
import os
import xml.etree.ElementTree as ET
import re

# IMPORTANT DO NOT REMOVE (BUT DO ADJUST IT ACC TO YOUR PATH)
osim.ModelVisualizer.addDirToGeometrySearchPaths("C:/OpenSim 4.5/Geometry")

action = "Shallow Squat"

base_path = f"C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Vicon Data/{action}"
setup_path = "C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Scripts/Setup_XMLs"
setup_path_relative = "../../../../Scripts/Setup_XMLs"


ik_path = os.path.join(base_path, 'IK').replace("\\", "/")
grf_path = os.path.join(base_path, 'ID', 'GRF').replace("\\", "/")
cmc_path = os.path.join(base_path, 'CMC').replace("\\", "/")
rra_path = os.path.join(base_path, 'RRA').replace("\\", "/")

setup_xml = os.path.join(setup_path,  "Boiler_XMLs", "analyze_setup.xml").replace("\\", "/")
task_xml = os.path.join(setup_path_relative, "Boiler_XMLs", "cmc_tasks.xml").replace("\\", "/")
actuator_xml = os.path.join(setup_path_relative, "Boiler_XMLs", "cmc_actuators.xml").replace("\\", "/")  
constraint_xml = os.path.join(setup_path_relative, "Boiler_XMLs", "cmc_constraints.xml").replace("\\", "/")  

ik_files = [f for f in os.listdir(ik_path) if f.endswith('.mot')]
grf_xmls = [f for f in os.listdir(grf_path) if f.endswith('.xml')]

print(f"IK files found {len(ik_files)}:", ik_files)
print(f"GRF files found {len(grf_xmls)}:", grf_xmls)

IK files found 9: ['1_Hanif_Shallow_2_5_patched.mot', '1_Hanif_Shallow_5_0_patched.mot', '1_Hanif_Shallow_7_5_patched.mot', '2_Hanif_Shallow_2_5_patched.mot', '2_Hanif_Shallow_5_0_patched.mot', '2_Hanif_Shallow_7_5_patched.mot', '3_Hanif_Shallow_2_5_patched.mot', '3_Hanif_Shallow_5_0_patched.mot', '3_Hanif_Shallow_7_5_patched.mot']
GRF files found 9: ['1_Hanif_Shallow_2_5_grf.xml', '1_Hanif_Shallow_5_0_grf.xml', '1_Hanif_Shallow_7_5_grf.xml', '2_Hanif_Shallow_2_5_grf.xml', '2_Hanif_Shallow_5_0_grf.xml', '2_Hanif_Shallow_7_5_grf.xml', '3_Hanif_Shallow_2_5_grf.xml', '3_Hanif_Shallow_5_0_grf.xml', '3_Hanif_Shallow_7_5_grf.xml']


In [9]:
# Generate CMC Result Folders if it doesn't exist

for ik_file in ik_files:
    trial_name = ik_file.replace('.mot', '')
    cmc_trial_path = os.path.join(cmc_path, trial_name).replace("\\", "/")
    if not os.path.exists(cmc_trial_path):
        os.makedirs(cmc_trial_path)
        print(f"Created RRA result folder for {trial_name} at {cmc_trial_path}")

In [10]:
# Reduce Residuals Batch Processing


for ik_file, grf_xml in zip(ik_files, grf_xmls):
    
    # Core RRA Logic

    trial_name = ik_file.replace('.mot', '')

    storage = osim.Storage(os.path.join(ik_path, ik_file).replace("\\", "/"))
    start_time = storage.getFirstTime()
    end_time = storage.getLastTime()
    print(f"IK File: {ik_file}, Start Time: {start_time}, End Time: {end_time}")

    results_folder = os.path.join(cmc_path, ik_file.replace('.mot', '')).replace("\\", "/")

    osim.Logger.removeFileSink()
    osim.Logger.addFileSink(os.path.join(results_folder, "opensim.log"))

    # Copy actuators.xml & tasks.xml
    # shutil.copy2(task_xml, results_folder)
    # shutil.copy2(actuator_xml, results_folder)
  
    tree = ET.parse(setup_xml)

    for elem in tree.iter('model_file'):
        elem.text = os.path.join(rra_path,trial_name, ik_file.replace(".mot","_model_adjusted.osim")).replace("\\","/")
    for elem in tree.iter('force_set_files'):
        elem.text = actuator_xml
    for elem in tree.iter('results_directory'):
        elem.text = results_folder
    for elem in tree.iter('initial_time'):
        elem.text = str(start_time)
    for elem in tree.iter('final_time'):
        elem.text = str(end_time)
    for elem in tree.iter('external_loads_file'):
        elem.text = os.path.join(grf_path, grf_xml).replace("\\", "/")
    for elem in tree.iter('desired_kinematics_file'):
        elem.text = os.path.join(ik_path, ik_file).replace("\\", "/")
    for elem in tree.iter('task_set_file'):
        elem.text = task_xml
    for elem in tree.iter('constraints_file'):
        elem.text = constraint_xml
    for elem in tree.iter('output_model_file'):
        elem.text = os.path.join(results_folder, ik_file.replace(".mot", "_model_adjusted.osim")).replace("\\", "/")

    output_xml = ik_file.replace('.mot', '_setup.xml')
    tree.write(os.path.join(results_folder, output_xml), encoding="UTF-8", xml_declaration=True)
  
    print(f"Results will be saved in: {results_folder}")
    rra_tool = osim.CMCTool(os.path.join(results_folder, output_xml).replace("\\", "/"))  
    rra_tool.run()

IK File: 1_Hanif_Shallow_2_5_patched.mot, Start Time: 0.2, End Time: 1.7


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Hanif/OneDrive - UNSW/T3 - 2025/Biomechanics of Human Body/Final Project/Scripts/Setup_XMLs/analyze_setup.xml'